In [ ]:
!pip install -q --upgrade google-generativeai langchain-google-genai python-dotenv

In [ ]:
import google.generativeai as genai

In [ ]:
# Used to securely store your API key
from google.colab import userdata
#dotenv package to load the API key
from dotenv import load_dotenv
from IPython.display import display
from IPython.display import Markdown
import textwrap

In [ ]:
GOOGLE_API_KEY=userdata.get('GeminiProKey')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
#Create a new .env file in the workspace and store the API key in it
!echo -e 'GOOGLE_API_KEY=XXXXXXXXXXXXXXXXXXXXXX' > .env

In [ ]:
!ls -a

In [ ]:
load_dotenv()


In [ ]:
#Create a helper function that will convert the markdown into nicely formatted text
def to_markdown(text):
  text = text.replace('•','*')
  return Markdown(textwrap.indent(text, '>', predicate=lambda _: True))

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)


In [ ]:
model = genai.GenerativeModel('gemini-pro')

In [ ]:
%%time
response = model.generate_content("What is the meaning of life?")

In [ ]:
response

In [ ]:
response.text

In [ ]:
to_markdown(response.text)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")

In [ ]:
result = llm.invoke("What is Mean Average Precision")

In [ ]:
to_markdown(result.content)

In [ ]:
!pip install langchain
!pip install pypdf
!pip install chromadb

In [ ]:
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [ ]:
!mkdir pdfs

In [ ]:
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load_and_split()

In [ ]:
print(data)


In [ ]:
context = "\n".join(str(p.page_content) for p in data)

In [ ]:
print("The total number of words in the context:", len(context))

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=200)
context = "\n\n".join(str(p.page_content) for p in data)

In [ ]:
texts = text_splitter.split_text(context)

In [ ]:
print(len(texts))


In [ ]:
texts[0]

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [ ]:
vector_index = Chroma.from_texts(texts, embeddings).as_retriever()

In [ ]:
question = "What are oppertunities and challenges for machine learning in material science"
docs = vector_index.get_relevant_documents(question)

In [ ]:
docs

In [ ]:
prompt_template = """
  Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
  provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
  Context:\n {context}?\n
  Question: \n{question}\n

  Answer:
"""

prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3)

In [ ]:
chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

In [ ]:
response = chain(
    {"input_documents":docs, "question": question}
    , return_only_outputs=True)


In [ ]:
response